# Introduction :
En Île-de-France, tandis qu'au temps de la révolution industrielle, les ménages les plus modestes qui possédaient des logements à proximité des usines étaient largement plus exposés à la pollution de l'air que les ménages aisés, nous nous sommes demandé si cette corrélation persistait toujours de nos jours. Autrement dit, nous nous sommes questionnés sur l'existence d'une corrélation entre l'exposition à la pollution de l'air et le niveau de vie des individus en Île-de-France. 

La pertinence de cette question provient d'une supposition paradoxale : d'une part, on peut supposer a priori que les ménages les plus modestes sont ceux qui possèdent des logements à proximité des axes routiers et donc exposés à un certain type de pollution, notamment au dioxyde d'azote (no2) ; d'autre part, on peut supposer a priori que les ménages les plus aisés sont ceux qui possèdent des logements en plein centre ville, où là aussi, la pollution liée au trafic routier semble plus importante. La littérature confirme l'existence de ce paradoxe dans certaines zones géographiques. Ainsi, Padilla et al. (2014) met en évidence dans son étude "Air quality and social deprivation in four French metropolitan areas – A localized spatiotemporal environmental inequality analysis" l'existence d'un paradoxe parisien où les quartiers les plus favorisés (SES élevé) sont souvent exposés à des concentrations moyennes de no2 plus élevées que les quartiers pauvres (effet de la densité urbaine).


Dans un premier temps, nous avons extrait et nettoyé les données de niveau de vie et de pollution de l’air. Les données de niveau de vie ont été récupérées à partir des bases de l’INSEE, tandis que les données de pollution atmosphérique sont plus difficiles d’accès. Nous avons donc d’abord procédé à du web scraping afin d’extraire les données de pollution au niveau communal depuis le site d’Airparif, puis utilisé l’API d’Airparif pour obtenir des données à une échelle infracommunale. 

Dans un second temps, nous avons analysé les données aux échelles communale et infracommunale afin de répondre à la question centrale de ce projet : existe-t-il un lien entre le niveau de vie des populations et leur exposition à la pollution de l’air en Île-de-France, et ce lien varie-t-il selon les caractéristiques urbaines et spatiales des territoires, notamment la proximité des axes routiers majeurs ?




# Data collection & cleaning

Ce notebook décrit :
1) la récupération des données (Airparif via webscraping, INSEE via CSV),
2) les contrôles de cohérence,
3) le nettoyage nécessaire,
4) la construction d’un jeu de données exploitable pour l’analyse.
5) la récupération de données plus précises (Airparif via api, Insee via shp)

## Step 0 — Environnement de travail et organisation des fichiers

Les analyses sont réalisées avec **Python 3 via Anaconda**.

Afin de garantir la reproductibilité du projet, les données sont organisées selon la structure suivante :
- `data/raw/` : données **brutes**, non modifiées ;
- `data/processed/` : données nettoyées et prêtes pour l’analyse.

Toutes les étapes de nettoyage sont réalisées à partir des données brutes, conformément aux consignes.




In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

# Définition des chemins de données et création des dossiers nécessaires
DATA_DIR = Path("../data")
RAW_DIR = DATA_DIR / "raw"
PROC_DIR = DATA_DIR / "processed"

RAW_DIR.mkdir(parents=True, exist_ok=True)
PROC_DIR.mkdir(parents=True, exist_ok=True)

# Option d'affichage pour faciliter l'exploration des DataFrames
pd.set_option("display.max_columns", 200)


## Fiabilité des sources

- **Airparif** : organisme officiel de surveillance de la qualité de l’air en Île-de-France. Les valeurs publiées proviennent de mesures et de modèles reconnus → source institutionnelle.
- **INSEE (Filosofi 2019 / données niveau de vie)** : source officielle statistique française, largement utilisée en recherche → source fiable.

Limites :
- Airparif : l’accès est via une interface web → scraping sensible aux changements de page.
- INSEE : niveau communal → possibles valeurs manquantes / agrégation selon communes.


## Step 1.1 — Webscraping Airparif (données pollution)

Nous avons récupéré les indicateurs de pollution (PM10, PM2.5, NO2) sur le site Airparif pour l’année 2019.
Le scraping automatise la sélection de l’année et la recherche de chaque commune, puis extrait la valeur affichée.


Attention : Le webscraping nécessite la bibliothèque Selenium qui elle même nécessite l'installation de Chromium. Sur Onyxia, l'installation de chromium se fait selon ces commandes tirées du site de Lino Galiana. Quelques ajustements ou mises à jours sont parfois nécessaires sur d'autres environnements.

In [2]:
"""
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb -O /tmp/chrome.deb
!sudo apt-get update
!sudo -E apt-get install -y /tmp/chrome.deb
!pip install chromedriver-autoinstaller selenium

import chromedriver_autoinstaller
path_to_web_driver = chromedriver_autoinstaller.install()
"""

'\n!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb -O /tmp/chrome.deb\n!sudo apt-get update\n!sudo -E apt-get install -y /tmp/chrome.deb\n!pip install chromedriver-autoinstaller selenium\n\nimport chromedriver_autoinstaller\npath_to_web_driver = chromedriver_autoinstaller.install()\n'

In [3]:
"""
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# réglages des options du navigateur chrome
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
"""

"\nfrom selenium import webdriver\nfrom selenium.webdriver.common.keys import Keys\nfrom selenium.webdriver.chrome.service import Service\nfrom selenium.webdriver.common.by import By\nfrom selenium.webdriver.support.ui import WebDriverWait\nfrom selenium.webdriver.support import expected_conditions as EC\nimport time\nimport pandas as pd\n\n# réglages des options du navigateur chrome\nchrome_options = webdriver.ChromeOptions()\nchrome_options.add_argument('--headless')\nchrome_options.add_argument('--no-sandbox')\nchrome_options.add_argument('--disable-dev-shm-usage')\n"

In [4]:
#lancement du navigateur Chrome
"""
service = Service(executable_path=path_to_web_driver)

browser = webdriver.Chrome(
    service=service,
    options=chrome_options
)
"""

'\nservice = Service(executable_path=path_to_web_driver)\n\nbrowser = webdriver.Chrome(\n    service=service,\n    options=chrome_options\n)\n'

Une fois les différents réglages réalisés et le webdriver chrome lancé, nous pouvons nous rendre sur la page web Airparif "https://www.airparif.fr/surveiller-la-pollution/bilans-et-cartes-annuels-de-pollution", qui regroupent les données annualisées de pollution en Ile de France pour trois polluants différents : 
-le pm10 qui correspondant aux particules fines ayant un diamètre inférieur à 10 micromètre. Elles représentent un risque mineur pour la santé humaine. Elles proviennent essentiellement de la poussière des chantiers, du labour ou de l'usure des pneus et de freins de véhicules.
-le pm25 qui correspondent aux particules fines ayant un diamètre inférieur à 2.5 micromètre. Contrairement aux précédentes, ces particules peuvent passer dans le sang par leur petit diamètre et elles représentent alors un risque majeur pour la santé humaine. Elles proviennent essentiellement des moteurs thermiques et du chauffage au bois.
-le no2 qui correspondent au gaz de dioxyde d'azote surtout émise par le trafic routier des diesels. Elles représentent un risque mineur pour la santé humaine.

In [5]:
#chargement de la page airparif
"""
browser.get('https://www.airparif.fr/surveiller-la-pollution/bilans-et-cartes-annuels-de-pollution')
time.sleep(10)
"""

"\nbrowser.get('https://www.airparif.fr/surveiller-la-pollution/bilans-et-cartes-annuels-de-pollution')\ntime.sleep(10)\n"

In [6]:
#choix de l'année 2019
"""
année="2019"
from selenium.webdriver.support.ui import Select
wait = WebDriverWait(browser, 10)
element_menu = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="date-filter"]')))
menu_deroulant = Select(element_menu)
menu_deroulant.select_by_visible_text(année)

time.sleep(3)
"""

'\nannée="2019"\nfrom selenium.webdriver.support.ui import Select\nwait = WebDriverWait(browser, 10)\nelement_menu = wait.until(EC.element_to_be_clickable((By.XPATH,\'//*[@id="date-filter"]\')))\nmenu_deroulant = Select(element_menu)\nmenu_deroulant.select_by_visible_text(année)\n\ntime.sleep(3)\n'

Après avoir chargé la page et l'année correspondante, nous pouvons extraire les données par webscraping. L'algorithme suivant reproduit donc exactement le comportement d'un utilisateur sur le site:
-nous parcourons une liste de commune donnée en entrée;
-nous écrivons le nom de chaque commune dans la barre de recherche, puis nous choisissons la première suggestion qui apparaît dans la barre de recherche qui correspond le plus souvent à la commune choisie (On procède de la sorte, car le bouton "search" ne fonctionne pas pour cette carte);
-on temporise le temps que la page charge, puis on récupère la valeur de pollution de sortie;
-on réitère ce parcourt pour les deux autres polluants.

In [7]:
#localisation de la barre des éléments : attention rajouter une intitialisation forcée à pm10
"""
from selenium.common.exceptions import StaleElementReferenceException

def fonction_taux_de_particule(l):
    search = browser.find_element("xpath",'//*[@id="carte-annuelle-panel-info"]//input')
    wait = WebDriverWait(browser, 3, ignored_exceptions=(StaleElementReferenceException,))
    valeur=[{"Commune" : e ,"pm10":None , "pm25":None , "no2":None} for e in l]
    liste_polluant=["pm10","pm25","no2"]
    bouton_polluant = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"a[data-pid={liste_polluant[0]}]")))
    browser.execute_script("arguments[0].click();", bouton_polluant)
    print(f"Clic forcé sur le {liste_polluant[0]} effectué via JavaScript.")
    time.sleep(5)
    for j in range(len(liste_polluant)):
        #Remarque:initialement on se situe sur le pm10 par défaut
        polluant=liste_polluant[j]
        if j <=1:
            polluant_suivant=liste_polluant[j+1]
        else:
            polluant_suivant=None
        for i in range(len(l)):
            #1/la barre de recherche
            search.clear()
            search.send_keys(l[i])  
            #2/la barre de suggestion
            try:
                suggestion_a_cliquer= wait.until(EC.element_to_be_clickable((By.CLASS_NAME,"pac-item")))
                suggestion_a_cliquer.click()
                time.sleep(0.2)
            #3/le taux de particules dans l'air
                try:
                    taux_de_particules=wait.until(EC.visibility_of_element_located((By.XPATH,'//*[@id="current-params"]/strong')))
                    valeur[i][polluant]=taux_de_particules.text
                except Exception as e:
                    print(f"Une erreur 1 s'est produite pour la commune {l[i]} et le polluant {liste_polluant[j]} -> attribution None aux valeurs")
            except Exception as e:
                print(f"Une erreur 2 s'est produite pour la commune {l[i]} et le polluant {liste_polluant[j]} -> attribution None aux valeurs")
        #chargement page nouveau polluant
        if polluant_suivant !=None:
            bouton_polluant = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"a[data-pid={polluant_suivant}]")))
            browser.execute_script("arguments[0].click();", bouton_polluant)
            print(f"Clic forcé sur le {polluant_suivant} effectué via JavaScript.")
            time.sleep(5)
    return(valeur)

"""

        

'\nfrom selenium.common.exceptions import StaleElementReferenceException\n\ndef fonction_taux_de_particule(l):\n    search = browser.find_element("xpath",\'//*[@id="carte-annuelle-panel-info"]//input\')\n    wait = WebDriverWait(browser, 3, ignored_exceptions=(StaleElementReferenceException,))\n    valeur=[{"Commune" : e ,"pm10":None , "pm25":None , "no2":None} for e in l]\n    liste_polluant=["pm10","pm25","no2"]\n    bouton_polluant = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"a[data-pid={liste_polluant[0]}]")))\n    browser.execute_script("arguments[0].click();", bouton_polluant)\n    print(f"Clic forcé sur le {liste_polluant[0]} effectué via JavaScript.")\n    time.sleep(5)\n    for j in range(len(liste_polluant)):\n        #Remarque:initialement on se situe sur le pm10 par défaut\n        polluant=liste_polluant[j]\n        if j <=1:\n            polluant_suivant=liste_polluant[j+1]\n        else:\n            polluant_suivant=None\n        for i in range(len(l)):\n

Avec la fonction précédente, nous pouvons récupérer les valeurs de pollution de chaque commune sous la forme d'un dataframe. Par précaution, nous sauvegardons les données au fur et à mesure de l'avancée de la fonction.

In [8]:
#extraction par paquet de 50
"""
def extraction_donnee(l):
    nombre2paquet=len(l)//50
    reste=len(l)-nombre2paquet
    decoupage=[(i+1)*50 for i in range(nombre2paquet)]
    if reste!=0:
        decoupage.append(reste)
    inf=0
    valeur=[]
    for j in decoupage:
        paquet=l[inf:j]
        v=fonction_taux_de_particule(paquet)
        valeur.extend(v)
        print(f"paquet {inf} à {j} traité -> paquet suivant")
        df=pd.DataFrame(valeur) 
        df.to_csv(f"donnees_commune_pollution_paquet{j}.csv",index=False)
        inf=j
    return(pd.DataFrame(valeur))
"""

'\ndef extraction_donnee(l):\n    nombre2paquet=len(l)//50\n    reste=len(l)-nombre2paquet\n    decoupage=[(i+1)*50 for i in range(nombre2paquet)]\n    if reste!=0:\n        decoupage.append(reste)\n    inf=0\n    valeur=[]\n    for j in decoupage:\n        paquet=l[inf:j]\n        v=fonction_taux_de_particule(paquet)\n        valeur.extend(v)\n        print(f"paquet {inf} à {j} traité -> paquet suivant")\n        df=pd.DataFrame(valeur) \n        df.to_csv(f"donnees_commune_pollution_paquet{j}.csv",index=False)\n        inf=j\n    return(pd.DataFrame(valeur))\n'

Nous pouvons désormais faire appel à la fonction, à partir de la liste des communes d'Ile de France, téléchargeable sur le site datagouv:

In [9]:
"""
import requests
import os

ROOT_DIR = Path().absolute().parent
dossier_data = ROOT_DIR / "data"
destination = os.path.join(dossier_data, "liste_communes_idf.csv")
reponse=requests.get("https://www.data.gouv.fr/api/1/datasets/r/91c0bdc4-0a5b-4ac8-950e-64a1ec207957")

if reponse.status_code == 200 :
    print("téléchargement réussi -> décompression")
    with open(destination,"wb") as f:
        f.write(reponse.content)

df_communes=pd.read_csv(destination, sep=";")
os.remove(destination)
df_communes = df_communes.sort_values(by="nomcom", ascending=True)
df_communes["nomcom"].to_csv(destination, index =False)
"""


'\nimport requests\nimport os\n\nROOT_DIR = Path().absolute().parent\ndossier_data = ROOT_DIR / "data"\ndestination = os.path.join(dossier_data, "liste_communes_idf.csv")\nreponse=requests.get("https://www.data.gouv.fr/api/1/datasets/r/91c0bdc4-0a5b-4ac8-950e-64a1ec207957")\n\nif reponse.status_code == 200 :\n    print("téléchargement réussi -> décompression")\n    with open(destination,"wb") as f:\n        f.write(reponse.content)\n\ndf_communes=pd.read_csv(destination, sep=";")\nos.remove(destination)\ndf_communes = df_communes.sort_values(by="nomcom", ascending=True)\ndf_communes["nomcom"].to_csv(destination, index =False)\n'

In [10]:
# attention, l'exécution du code pour la liste de commune prend quelques heures
#df2=extraction_donnee(liste_commune)

⚠️ Le scraping peut être long. Nous sauvegardons systématiquement la sortie brute dans `../data/raw/airparif_pollution_raw.csv`.

In [11]:
# Chargement des données de pollution Airparif (données brutes)

poll_path = RAW_DIR / "airparif_polution_raw.csv"
poll_raw = pd.read_csv(poll_path)

poll_raw.head()


,Commune,pm10,pm25,no2
0,ABBEVILLE LA RIVIERE,15.0,8.0,8.0
1,ABLEIGES,18.0,9.0,13.0
2,ABLIS,15.0,8.0,13.0
3,ABLON SUR SEINE,17.0,10.0,24.0
4,ACHERES,19.0,10.0,21.0


In [12]:
poll_raw["Commune"].duplicated().sum()
poll_raw.shape

(1300, 4)

## Step 1.2 — Récupération des données INSEE (format CSV)

Les données INSEE sont récupérées sous forme de fichiers CSV directement depuis le site de l’INSEE.  
Les fichiers sont stockés **sans modification** dans le dossier `data/raw/`.

Le séparateur utilisé est le point-virgule (`;`), conformément au format standard des fichiers INSEE.


In [13]:
filo_path = RAW_DIR / "FILO2019_DISP_COM.csv"  # adapte au nom exact
filo_raw = pd.read_csv(filo_path, sep=";")
filo_raw.head()



/tmp/ipykernel_6195/4015194515.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  filo_raw = pd.read_csv(filo_path, sep=";")


,CODGEO,NBMEN19,NBPERS19,NBUC19,Q119,Q219,Q319,Q3_Q1,D119,D219,D319,D419,D619,D719,D819,D919,RD,S80S2019,GI19,PACT19,PTSA19,PCHO19,PBEN19,PPEN19,PPAT19,PPSOC19,PPFAM19,PPMINI19,PPLOGT19,PIMPOT19,AGE1Q119,AGE1Q219,AGE1Q319,AGE1Q3_Q1,AGE1D119,AGE1D219,AGE1D319,AGE1D419,AGE1D619,AGE1D719,AGE1D819,AGE1D919,AGE1RD,AGE1S80S2019,AGE1GI19,AGE1PACT19,AGE1PTSA19,AGE1PCHO19,AGE1PBEN19,AGE1PPEN19,AGE1PPAT19,AGE1PPSOC19,AGE1PPFAM19,AGE1PPMINI19,AGE1PPLOGT19,AGE1PIMPOT19,AGE2Q119,AGE2Q219,AGE2Q319,AGE2Q3_Q1,AGE2D119,AGE2D219,AGE2D319,AGE2D419,AGE2D619,AGE2D719,AGE2D819,AGE2D919,AGE2RD,AGE2S80S2019,AGE2GI19,AGE2PACT19,AGE2PTSA19,AGE2PCHO19,AGE2PBEN19,AGE2PPEN19,AGE2PPAT19,AGE2PPSOC19,AGE2PPFAM19,AGE2PPMINI19,AGE2PPLOGT19,AGE2PIMPOT19,AGE3Q119,AGE3Q219,AGE3Q319,AGE3Q3_Q1,AGE3D119,AGE3D219,AGE3D319,AGE3D419,AGE3D619,AGE3D719,AGE3D819,AGE3D919,AGE3RD,AGE3S80S2019,AGE3GI19,AGE3PACT19,AGE3PTSA19,AGE3PCHO19,...,OPR3D119,OPR3D219,OPR3D319,OPR3D419,OPR3D619,OPR3D719,OPR3D819,OPR3D919,OPR3RD,OPR3S80S2019,OPR3GI19,OPR3PACT19,OPR3PTSA19,OPR3PCHO19,OPR3PBEN19,OPR3PPEN19,OPR3PPAT19,OPR3PPSOC19,OPR3PPFAM19,OPR3PPMINI19,OPR3PPLOGT19,OPR3PIMPOT19,OPR4Q119,OPR4Q219,OPR4Q319,OPR4Q3_Q1,OPR4D119,OPR4D219,OPR4D319,OPR4D419,OPR4D619,OPR4D719,OPR4D819,OPR4D919,OPR4RD,OPR4S80S2019,OPR4GI19,OPR4PACT19,OPR4PTSA19,OPR4PCHO19,OPR4PBEN19,OPR4PPEN19,OPR4PPAT19,OPR4PPSOC19,OPR4PPFAM19,OPR4PPMINI19,OPR4PPLOGT19,OPR4PIMPOT19,OPR5Q119,OPR5Q219,OPR5Q319,OPR5Q3_Q1,OPR5D119,OPR5D219,OPR5D319,OPR5D419,OPR5D619,OPR5D719,OPR5D819,OPR5D919,OPR5RD,OPR5S80S2019,OPR5GI19,OPR5PACT19,OPR5PTSA19,OPR5PCHO19,OPR5PBEN19,OPR5PPEN19,OPR5PPAT19,OPR5PPSOC19,OPR5PPFAM19,OPR5PPMINI19,OPR5PPLOGT19,OPR5PIMPOT19,OPR6Q119,OPR6Q219,OPR6Q319,OPR6Q3_Q1,OPR6D119,OPR6D219,OPR6D319,OPR6D419,OPR6D619,OPR6D719,OPR6D819,OPR6D919,OPR6RD,OPR6S80S2019,OPR6GI19,OPR6PACT19,OPR6PTSA19,OPR6PCHO19,OPR6PBEN19,OPR6PPEN19,OPR6PPAT19,OPR6PPSOC19,OPR6PPFAM19,OPR6PPMINI19,OPR6PPLOGT19,OPR6PIMPOT19
0,1001,322,819,542.7,NaN,23970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1002,113,263,177.2,NaN,23640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1004,6711,15040,10296.8,14940.0,20510,27310.0,12370.0,11080.0,13800.0,16110.0,18340.0,22940.0,25670.0,29110.0,35230.0,3.2,4.2,0.290,70.1,63.4,3.1,3.6,25.6,12.3,7.7,2.8,3.1,1.8,-15.7,14200.0,18850.0,23850.0,9640.0,10140.0,12790.0,15040.0,17270.0,20330.0,22830.0,25060.0,28740.0,2.8,3.1,0.218,101.0,93.7,5.5,1.8,0.5,1.7,10.1,2.7,4.5,2.9,-13.3,13620.0,18690.0,25320.0,11700.0,10510.0,12490.0,14580.0,16590.0,21000.0,23670.0,26850.0,31660.0,3.0,

## Step 1.3 — Contrôles de cohérence des données brutes

Avant toute transformation, nous réalisons des contrôles simples mais essentiels afin de vérifier la cohérence des données brutes issues d’Airparif et de l’INSEE.

Ces contrôles portent notamment sur :
- les dimensions des jeux de données (nombre d’observations et de variables) ;
- la présence de doublons ;
- la part de valeurs manquantes par variable.

Ils permettent d’identifier d’éventuels problèmes (variables très incomplètes, doublons, erreurs de structure) avant de procéder au nettoyage.


In [14]:
def quick_checks(df, name):
    print(f"--- {name} ---")
    print("shape:", df.shape)
    print("duplicates rows:", df.duplicated().sum())
    print("missing % (top 10):")
    miss = (df.isna().mean().sort_values(ascending=False) * 100).head(10)
    display(miss)

quick_checks(poll_raw, "Airparif raw")
quick_checks(filo_raw, "INSEE raw")


--- Airparif raw ---
shape: (1300, 4)
duplicates rows: 0
missing % (top 10):


pm10       2.769231
pm25       2.615385
no2        2.538462
Commune    0.000000
dtype: float64

--- INSEE raw ---
shape: (31361, 732)
duplicates rows: 0
missing % (top 10):


OPR6PIMPOT19    99.996811
OPR6PPMINI19    99.996811
OPR6PPLOGT19    99.996811
OPR6PPEN19      99.996811
OPR6PPAT19      99.996811
OPR6PACT19      99.996811
OPR6PTSA19      99.996811
OPR6PBEN19      99.996811
OPR6PPFAM19     99.996811
OPR6PCHO19      99.996811
dtype: float64

### Interprétation des contrôles de cohérence

Les données de pollution issues d’Airparif comportent environ 1300 observations correspondant à des communes, avec un nombre limité de variables.  
Aucun doublon n’est détecté et la proportion de valeurs manquantes reste faible (inférieure à 3 %), ce qui est satisfaisant pour ce type de données environnementales.

Les données socio-économiques de l’INSEE (Filosofi) sont beaucoup plus volumineuses et contiennent un grand nombre de variables.  
Certaines d’entre elles présentent une proportion très élevée de valeurs manquantes, ce qui est attendu compte tenu du niveau de détail et de la diversité des indicateurs proposés.

Ces constats justifient la sélection d’un nombre restreint de variables pertinentes pour la suite de l’analyse.


In [15]:
#On vérifie les colonnes de pollution et leur cohérence
for col in ["pm10", "pm25", "no2"]:
    if col in poll_raw.columns:
        s = pd.to_numeric(poll_raw[col], errors="coerce")
        print(col, "min=", s.min(), "max=", s.max(), "missing=", s.isna().mean())


pm10 min= 14.0 max= 28.0 missing= 0.027692307692307693
pm25 min= 8.0 max= 15.0 missing= 0.026153846153846153
no2 min= 5.0 max= 54.0 missing= 0.025384615384615384


## Step 1.4 — Sélection des variables socio-économiques pertinentes (INSEE)

Les données Filosofi contiennent un grand nombre de variables décrivant la distribution des niveaux de vie.  
Dans le cadre de ce projet, nous retenons un nombre limité de variables pertinentes afin de caractériser le niveau de vie et les inégalités au niveau communal.

Les variables sélectionnées sont :
- **Q219** : niveau de vie médian ;
- **D119** : premier décile du niveau de vie, utilisé comme indicateur de pauvreté ;
- **GI19** : indice de Gini, mesurant les inégalités de revenus ;
- **NBPERS19** : population communale ;
- **CODGEO** : code géographique de la commune (clé de jointure).

Ces variables permettent de capturer à la fois le niveau de vie moyen, la situation des ménages les plus modestes et le degré d’inégalité au sein des communes.


In [16]:
# Sélection des variables INSEE pertinentes
cols_insee = ["CODGEO", "NBPERS19", "Q219", "D119", "GI19"]
cols_insee = [c for c in cols_insee if c in filo_raw.columns]

filo = filo_raw[cols_insee].copy()

# Harmonisation du code géographique
filo["CODGEO"] = filo["CODGEO"].astype(str).str.zfill(5)

filo.head()


,CODGEO,NBPERS19,Q219,D119,GI19
0,01001,819,23970,NaN,NaN
1,01002,263,23640,NaN,NaN
2,01004,15040,20510,11080.0,0.290
3,01005,1823,24050,NaN,NaN
4,01007,2853,23380,14360.0,0.206


### Construction d’un indicateur de pauvreté relative

Afin de caractériser la situation des ménages les plus modestes relativement au niveau de vie global de la commune, nous construisons un indicateur de pauvreté relative défini comme le rapport entre le premier décile et le niveau de vie médian.


In [17]:
filo["poverty_index"] = filo["D119"] / filo["Q219"]

filo[["Q219", "D119", "poverty_index"]].describe()


,Q219,D119,poverty_index
count,31361.000000,5336.000000,5336.000000
mean,22065.303083,13004.778861,0.575235
std,3042.079487,2072.141838,0.052009
min,11730.000000,6540.000000,0.242549
25%,20160.000000,11490.000000,0.546400
50%,21610.000000,13040.000000,0.579230
75%,23450.000000,14450.000000,0.609444
max,51720.000000,21200.000000,0.709871


## Step 1.5 — Harmonisation des clés géographiques (communes)

Les données de pollution issues d’Airparif ne contiennent pas directement le **code géographique INSEE (CODGEO)** des communes, mais uniquement le **nom de la commune**.

À l’inverse, les données socio-économiques de l’INSEE (Filosofi) sont identifiées par le **CODGEO**, qui constitue la clé la plus fiable pour effectuer une jointure au niveau communal.

Afin de pouvoir croiser ces deux sources, nous utilisons un fichier de correspondance INSEE associant les **noms de communes** à leur **code géographique (CODGEO)**.

Cette étape nécessite un travail d’harmonisation des noms de communes, notamment pour :
- les abréviations (ex. *St* → *Saint*) ;
- les différences d’orthographe ou de typographie ;
- les caractères spéciaux et accents.


In [18]:
# Chargement du fichier de correspondance communes (INSEE)
communes_path = RAW_DIR / "v_commune_2025.csv"
communes = pd.read_csv(communes_path)

communes.head()


,TYPECOM,COM,REG,DEP,CTCD,ARR,TNCC,NCC,NCCENR,LIBELLE,CAN,COMPARENT
0,COM,01001,84.0,01,01D,012,5,ABERGEMENT CLEMENCIAT,Abergement-Clémenciat,L'Abergement-Clémenciat,0108,NaN
1,COM,01002,84.0,01,01D,011,5,ABERGEMENT DE VAREY,Abergement-de-Varey,L'Abergement-de-Varey,0101,NaN
2,COM,01004,84.0,01,01D,011,1,AMBERIEU EN BUGEY,Ambérieu-en-Bugey,Ambérieu-en-Bugey,0101,NaN
3,COM,01005,84.0,01,01D,012,1,AMBERIEUX EN DOMBES,Ambérieux-en-Dombes,Ambérieux-en-Dombes,0122,NaN
4,COM,01006,84.0,01,01D,011,1,AMBLEON,Ambléon,Ambléon,0104,NaN


In [19]:
# Filtrer pour ne garder que les communes d'Île-de-France (REG = 11)
communes = communes[communes["REG"] == 11].copy()


communes = communes[["COM", "NCC"]]

communes = communes.rename(columns={
    "COM": "CODGEO",
    "NCC": "Commune"
})

In [20]:
communes["Commune"].duplicated().sum()
communes["Commune"].value_counts().head(20)


Commune
SAINT MARTIN DES CHAMPS    2
MONDREVILLE                2
MAROLLES EN BRIE           2
BLANDY                     2
FORGES LES BAINS           1
FORET SAINTE CROIX         1
FORET LE ROI               1
FONTENAY LE VICOMTE        1
FONTENAY LES BRIIS         1
FONTAINE LA RIVIERE        1
FLEURY MEROGIS             1
GIF SUR YVETTE             1
EVRY COURCOURONNES         1
ETRECHY                    1
ETIOLLES                   1
ETAMPES                    1
EPINAY SUR ORGE            1
EPINAY SOUS SENART         1
EGLY                       1
ECHARCON                   1
Name: count, dtype: int64

### Gestion des doublons dans la table de correspondance des communes

La table de correspondance INSEE peut contenir plusieurs lignes pour un même nom de commune, notamment en raison :
- de changements administratifs ;
- de différences de périmètre ;
- ou de la présence d’informations complémentaires non nécessaires pour la jointure.

Dans le cadre de ce projet, la jointure avec les données de pollution repose uniquement sur le **nom de la commune**, sans distinction supplémentaire.  
Afin d’éviter toute ambiguïté lors de la jointure (plusieurs codes géographiques associés à un même nom), nous conservons une seule occurrence par commune.

Cette opération permet de garantir une correspondance **univoque** entre le nom de la commune et le code géographique.


In [21]:
# Suppression des doublons afin de garantir une correspondance univoque
communes = communes.drop_duplicates(subset=["Commune"], keep="first")

# Vérification de l'unicité
communes["Commune"].is_unique


True

In [22]:
# Nettoyage des noms de communes pour faciliter la jointure
def clean_commune_name(s):
    s = s.upper()
    s = s.replace("-", " ")
    s = s.replace("ST ", "SAINT ")
    s = s.replace("STE ", "SAINTE ")
    s = s.strip()
    return s

poll_clean = poll_raw.copy()
poll_clean["Commune_clean"] = poll_clean["Commune"].apply(clean_commune_name)
communes["Commune_clean"] = communes["Commune"].apply(clean_commune_name)


In [23]:
poll_clean = poll_clean.merge(
    communes[["Commune_clean", "CODGEO"]],
    on="Commune_clean",
    how="left",
    validate="m:1"
)

poll_clean.head(10)

,Commune,pm10,pm25,no2,Commune_clean,CODGEO
0,ABBEVILLE LA RIVIERE,15.0,8.0,8.0,ABBEVILLE LA RIVIERE,91001
1,ABLEIGES,18.0,9.0,13.0,ABLEIGES,95002
2,ABLIS,15.0,8.0,13.0,ABLIS,78003
3,ABLON SUR SEINE,17.0,10.0,24.0,ABLON SUR SEINE,94001
4,ACHERES,19.0,10.0,21.0,ACHERES,78005
5,ACHERES LA FORET,14.0,8.0,8.0,ACHERES LA FORET,77001
6,ADAINVILLE,20.0,8.0,9.0,ADAINVILLE,78006
7,AIGREMONT,17.0,9.0,17.0,AIGREMONT,78007
8,AINCOURT,16.0,9.0,9.0,AINCOURT,95008
9,ALFORTVILLE,19.0,12.0,33.0,ALFORTVILLE,94002


In [24]:
# Extraire les communes sans code INSEE après la jointure
communes_non_appariees = (
    poll_clean.loc[poll_clean["CODGEO"].isna(), "Commune"]
    .drop_duplicates()
    .sort_values()
)

communes_non_appariees


257                  CHENOISE
346        DAUMAZAN SUR ARIZE
384                   ESCOSSE
392                      EVRY
395                     FABAS
                ...          
1157      STE CROIX VOLVESTRE
1184                TOURTOUSE
1189                TREMOULET
1195                   UNZENT
1263    VILLENEUVE DU PAREAGE
Name: Commune, Length: 162, dtype: object

In [25]:
# Nombre de communes sans code INSEE après la jointure
n_missing = poll_clean["CODGEO"].isna().sum()
total = poll_clean.shape[0]

print(f"Communes sans CODGEO : {n_missing} sur {total}")
print(f"Proportion : {round(n_missing / total * 100, 2)} %")


Communes sans CODGEO : 162 sur 1300
Proportion : 12.46 %


Après harmonisation des noms de communes et jointure avec la table de correspondance INSEE,  
162 communes ne peuvent pas être associées à un code géographique, soit environ 13 % des observations initiales.

Ces non-appariements s’expliquent principalement par :
- des différences résiduelles d’orthographe ou de typographie ;
- des communes hors du périmètre couvert par la table de correspondance ;
- des particularités administratives.

Ces observations sont exclues de la suite de l’analyse afin de garantir la cohérence du jeu de données final.


In [26]:
poll_clean["CODGEO"] = poll_clean["CODGEO"]
poll_clean.head(4)

poll_clean_geo = poll_clean.dropna(subset=["CODGEO"])


Les données de pollution et de niveau de vie sont fusionnées à l’aide du code INSEE des communes (CODGEO),
qui constitue une clé d’identification unique et standardisée.

Une jointure interne (`inner join`) est utilisée afin de conserver uniquement les communes
pour lesquelles l’ensemble des informations nécessaires est disponible.
Ce choix garantit la cohérence et la comparabilité des analyses ultérieures.


In [27]:
df_final = poll_clean_geo.merge(
    filo,
    on="CODGEO",
    how="inner"
)

df_final.head(5)


,Commune,pm10,pm25,no2,Commune_clean,CODGEO,NBPERS19,Q219,D119,GI19,poverty_index
0,ABBEVILLE LA RIVIERE,15.0,8.0,8.0,ABBEVILLE LA RIVIERE,91001,336,24590,NaN,NaN,NaN
1,ABLEIGES,18.0,9.0,13.0,ABLEIGES,95002,1153,28270,NaN,NaN,NaN
2,ABLIS,15.0,8.0,13.0,ABLIS,78003,3563,25970,16030.0,0.208,0.617251
3,ABLON SUR SEINE,17.0,10.0,24.0,ABLON SUR SEINE,94001,5741,22780,11530.0,0.270,0.506146
4,ACHERES,19.0,10.0,21.0,ACHERES,78005,22032,22260,12030.0,0.257,0.540431


In [28]:
print("Pollution géocodée :", poll_clean_geo.shape)
print("Filo sélectionné   :", filo.shape)
print("Base finale        :", df_final.shape)


Pollution géocodée : (1138, 6)
Filo sélectionné   : (31361, 6)
Base finale        : (1125, 11)


Après géocodage des données de pollution, 1 138 communes d’Île-de-France disposent
d’un code INSEE valide.  
Les données Filosofi couvrent l’ensemble des communes françaises (31 361 communes),
mais seules celles présentes dans les données de pollution sont conservées lors
de la jointure.

La base finale contient 1 125 communes, soit une perte de seulement 13 observations
par rapport aux données de pollution géocodées. Cette différence s’explique par
l’absence de certaines informations de niveau de vie pour quelques communes.
L’échantillon final reste donc très représentatif.


In [29]:
# Sauvegarde des données nettoyées pour les analyses suivantes
output_path = PROC_DIR / "df_final.csv"
df_final.to_csv(output_path, index=False)


Les données nettoyées sont sauvegardées localement dans le dossier `data/processed`
afin d’être réutilisées dans les notebooks d’analyse et de modélisation.
Ces fichiers ne sont pas versionnés sur GitHub.


## Step 2.1 — Récupération de données au niveau de carreaux de 200 m × 200 m

Après avoir récupéré les données au niveau communal, nous avons cherché à affiner l’analyse
en exploitant les données au **plus petit niveau géographique disponible**, à savoir des
carreaux de **200 mètres de côté**.

Les données de niveau de vie ont été récupérées directement sur le site de l’**INSEE**, au
format *shapefile*, à partir de la base  
*« Revenus, pauvreté et niveau de vie en 2019 – Données carroyées »*.

Afin de limiter la quantité de données manipulées en mémoire, nous avons restreint l’étude
aux carreaux situés en **Île-de-France**.

Pour chaque carreau, un appel a ensuite été effectué à l’API utilisée en arrière-plan par
la **carte interactive d’Airparif**, en fournissant les coordonnées géographiques (préalablement
converties dans le système requis) correspondant au centre de chaque carreau.

L’exécution complète de ces appels successifs à l’API ayant nécessité environ **9 heures**,
nous fournissons directement dans ce projet les **données de pollution déjà récupérées**,
afin de garantir la reproductibilité et la facilité d’exécution du notebook.


In [30]:
#installation de la librairie py7zr nécessaire pour déziper les données Insee
import sys
import subprocess

try:
    __import__("py7zr")
except ImportError:
    print(f"Installation de py7zr en cours...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "py7zr"])

Installation de py7zr en cours...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 76.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 93.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [py7zr]32m7/9 [backports.zstd]


In [31]:
from zipfile import ZipFile
import py7zr
import geopandas as gpd

La fonction suivante remplit plusieurs objectifs :

- récupération et décompression du jeu de données de l’INSEE  
  *« Revenus, pauvreté et niveau de vie en 2019 – Données carroyées »* ;
- sélection des données carroyées pour la région **Île-de-France**, en s’appuyant sur la
  liste des communes disponible sur *data.gouv.fr* et en réalisant une **jointure** entre
  les deux jeux de données ;
- intégration des données de **pollution de l’air**, préalablement récupérées via l’API,
  et construction d’un **dataframe final** combinant indicateurs de niveau de vie et
  mesures de pollution.


In [32]:
import requests
from zipfile import ZipFile
import os
import py7zr
import pandas as pd
import geopandas as gpd
from pathlib import Path


def telecharger_donnees():
    print("téléchargement des données (1 à 2 minutes)")
    ROOT_DIR = Path().absolute().parent
    dossier_data = ROOT_DIR / "data/raw"
    
    # --- téléchargement des données carroyées de niveau de vie Insee ---
    URL_INSEE_ZIP = "https://www.insee.fr/fr/statistiques/fichier/7655475/Filosofi2019_carreaux_200m_shp.zip"
    intermediaire_zip = "donnees.zip"
    reponse = requests.get(URL_INSEE_ZIP)
    if reponse.status_code == 200:
        print("téléchargement réussi -> décompression")
        with open(intermediaire_zip, "wb") as f:
            f.write(reponse.content)
        with ZipFile(intermediaire_zip, 'r') as zip_ref:
            zip_ref.extractall(dossier_data)
        chemin_fichier_7z = os.path.join(dossier_data, "Filosofi2019_carreaux_200m_shp.7z")
        print("decompression zip terminée -> deuxième décompression")
        with py7zr.SevenZipFile(chemin_fichier_7z, mode="r") as z:
            z.extractall(path=dossier_data)

        # suppression des deux fichiers zip:
        os.remove(os.path.join(os.getcwd(), intermediaire_zip))
        os.remove(chemin_fichier_7z)
    else:
        print(f"echec du télechargement code status : {reponse.status_code}")

    # --- suppression des fichiers inutiles de la réunion et de la martinique---
    extensions = ["dbf", "prj", "shp", "shx"]
    for e in extensions:
        try:
            os.remove(os.path.join(dossier_data, f"carreaux_200m_mart.{e}"))
        except Exception:
            print("erreur_suppresion_martinique")
        try:
            os.remove(os.path.join(dossier_data, f"carreaux_200m_reun.{e}"))
        except Exception:
            print("erreur_suppresion_reunion")
    print("suppression données inutiles martinique et réunion")

    # --- conversion en dataframe centré sur l'idf ---
    print("conversion des données de niveau de vie INSEE en un dataframe")
    df = gpd.read_file(os.path.join(dossier_data, "carreaux_200m_met.shp"))
    print("conversion terminée")
    
    # --- récupération des départements associés à lcog_geo ---
    url_base = "https://geo.api.gouv.fr/communes"
    parametres = {
        "codeRegion": "11",              # Île-de-France
        "type": "commune-actuelle,arrondissement-municipal",
        "fields": "nom,code,codeDepartement",  # Les colonnes voulues
        "format": "json"}
    response = requests.get(url_base, params=parametres)
    df2 = pd.DataFrame(response.json())

    # --- rightjoin ---
    df3 = pd.merge(df, df2, left_on="lcog_geo", right_on="code", how="right")
    print("récupération des données spécifiques à l'Ile-de-France")

    # --- récupération des données de pollution ---
    polluants = ["no2", "pm10", "pm25"]
    for p in polluants:
        df3[p] = pd.read_csv(os.path.join(dossier_data, f"donnees_pollution{p}_checkpoint.csv"))
    "ajout des valeurs de pollution au dataset sur le niveau de vie en Ile-de-France"

    # --- suppression des données téléchargées ---
    for e in extensions:
        try:
            os.remove(os.path.join(dossier_data, f"carreaux_200m_met.{e}"))
        except Exception:
            print("erreur_suppresion_métropole")
    print("téléchargement des données terminée")
    return (df3)

df=telecharger_donnees()

téléchargement des données (1 à 2 minutes)
téléchargement réussi -> décompression
decompression zip terminée -> deuxième décompression
suppression données inutiles martinique et réunion
conversion des données de niveau de vie INSEE en un dataframe
conversion terminée
récupération des données spécifiques à l'Ile-de-France
téléchargement des données terminée


In [33]:
ROOT_DIR = Path().absolute().parent
dossier_data = ROOT_DIR / "data/raw"
df.to_csv(os.path.join(dossier_data,"donnees_pollutions_carreaux"), index =False)

Nous fournissons également le code permettant de faire appel à l’API **Airparif**.  
Ce code n’est **pas exécuté automatiquement**, son exécution complète étant très longue
(plusieurs heures). Il est présenté à des fins de **transparence méthodologique** et
pourrait être intégré dans la section *Récupération des données de pollution*.

Ce code procède en plusieurs étapes :

- conversion des coordonnées géographiques dans le format requis par l’API Airparif,
  à savoir le **Lambert II étendu (EPSG:27572)** ;
- récupération des coordonnées \((x, y)\) du centre de chaque carreau, autour desquelles
  est construit un **carré de requête** afin d’interroger l’API ;
- définition d’une fonction permettant **d’extraire automatiquement la donnée d’intérêt**
  à partir du texte retourné par l’API ;
- récupération, pour chaque **polluant** et pour l’année **2019**, des données de pollution
  associées à chaque carreau, en intégrant plusieurs mesures de précaution :
  sauvegarde régulière des résultats intermédiaires, gestion des erreurs et
  **temps d’attente de 0,1 seconde** entre deux requêtes afin de ne pas surcharger le serveur.


In [34]:
# import de bibliothèques supplémentaires pour la gestion du temps et des expressions régulières
import re
from tqdm import tqdm
import time

In [35]:
# 1/conversion des coordonnées géographiques
# df3 = df3.to_crs(27572)

In [36]:
# 2/ récupération des positions des centres des carreaux
#df3["centre"] = df3.centroid
#df3["x_centre"] = df3["centre"].x
#df3["y_centre"] = df3["centre"].y

def build_bbox_string(row):
    x_min = row["x_centre"]-0.5
    x_max = row["x_centre"]+0.5
    y_min = row["y_centre"]-0.5
    y_max = row["y_centre"]+0.5
    return (f"{x_min},{y_min},{x_max},{y_max}")

#df3["bbox"] = df3.apply(build_bbox_string, axis=1)

In [37]:
# 3/Extraction des valeurs 
def extraire_pollution(texte):
    try:
        # On cherche les chiffres après 'GRAY_INDEX = '
        match = re.search(r"GRAY_INDEX\s*=\s*([\d.]+)", texte)
        return float(match.group(1)) if match else None
    except Exception:
        return None

In [38]:
#récupération des données de pollution (attention, l'exécution de cette fonction prends plusieurs heures)

def appel_api_pollution(polluant):
    if polluant not in ["pm10", "pm25", "no2"]:
        return ("Le polluant doit être pm10, pm25 ou no2")
    else:
        resultats = []
        BATCH_SIZE = 100  # On sauvegarde toutes les 100 lignes
        ROOT_DIR = Path().absolute().parent
        dossier_data = ROOT_DIR / "data_carreaux"
        FICHIER_SAUVEGARDE = os.path.join(dossier_data, f"donnees_pollution{polluant}_checkpoint.csv")

        for index, row in tqdm(df3.iterrows(), total=len(df3)):
            # initialisation des paramètres d'appel à l'API
            params = {
                "SERVICE": "WMS",
                "VERSION": "1.1.1",
                "REQUEST": "GetFeatureInfo",
                "QUERY_LAYERS": f"Moyenne_annuelle:{polluant}",
                "LAYERS": f"Moyenne_annuelle:{polluant}",
                "INFO_FORMAT": "text/plain",
                "BBOX": row['bbox'],
                "WIDTH": 1,
                "HEIGHT": 1,
                "X": 0,
                "Y": 0,
                "SRS": "EPSG:27572",
                "TIME": "2019"
            }

            # tentative d'appel à l'API
            try:
                r = requests.get(url_base, params=params, timeout=10)
                if r.status_code == 200:
                    valeur = extraire_pollution(r.text)
                    resultats.append(valeur)
                else:
                    resultats.append(None)
            except Exception as e:
                resultats.append(None)

            # sauvegarde des données de pollution toutes les 100 mesures
            if (index + 1) % BATCH_SIZE == 0:
                    df_checkpoint = pd.DataFrame(resultats)
                # S'il existe déjà, on ajoute les données à la suite (mode='a' pour append)
                    mode = 'a' if os.path.exists(FICHIER_SAUVEGARDE) else 'w'
                    header = not os.path.exists(FICHIER_SAUVEGARDE)
                    
                    df_checkpoint.to_csv(FICHIER_SAUVEGARDE, mode=mode, header=header, index=False)
                    
                    # Vider la liste temporaire pour ne pas ré-écrire les mêmes données
                    resultats = []
            
            # Pause de sécurité
            time.sleep(0.1)

        # Récupération des dernières données de pollution
        if resultats:
            pd.DataFrame(resultats).to_csv(FICHIER_SAUVEGARDE, mode='a', header=False, index=False)
        print(resultats)

        # Ajout des données de pollution au dataframe
        df3[f'{polluant}_2019'] = pd.read_csv(FICHIER_SAUVEGARDE)


# Step 2.2 - Qualité des données infracommunalles

Pour ce qui est des valeurs manquantes, elles sont quasiment inexistantes (199 sur 60 085 soit environ 0.3% de lignes ayant au moins une valeur manquante) et elles proviennent en majorité d'une erreur de l'appel à l'API AirParif.

In [39]:
print(f"Il y a au total {len(df)} observations.")
nb_lignes_manquantes = df.isnull().any(axis=1).sum()
print(f"Il y a {nb_lignes_manquantes} lignes avec au moins une valeur manquante.")

Il y a au total 60085 observations.
Il y a 199 lignes avec au moins une valeur manquante.


Notons que le jeu de données obtenu ne couvre pas l’intégralité du territoire francilien. En effet, pour des raisons de secret statistique, l’INSEE ne diffuse pas les indicateurs de niveau de vie pour les zones comptant moins de 11 habitants. Certaines zones carroyées de 200 m × 200 m sont donc absentes de notre base.

Néanmoins, cette limitation reste relativement marginale : selon la documentation FILOSOFI 2019 de l’INSEE, environ 80 % de la population française réside dans des zones carroyées de 200 m × 200 m. L’échantillon utilisé demeure ainsi largement représentatif des espaces effectivement habités, en particulier dans un contexte urbain dense comme l’Île-de-France.


En somme, le dataset est de très bonne qualité, et ne nécessite pas en l'état de traitement particuliers.

On va également importer une base de données de routes en Ile de France (d'autoroutes aux départemenatles pour garder les grands axes principaux) qu'on va réutiliser pour mener une analyse plus profonde du lien entre niveau de vie et exposition à la pollution.

In [41]:
#installation de la librairie py7zr nécessaire pour déziper les données Insee
import sys
import subprocess

try:
    __import__("osmnx")
except ImportError:
    print(f"Installation de osmnx en cours...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "osmnx"])

Installation de osmnx en cours...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 82.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [osmnx]kx]


In [42]:
import osmnx as ox
import os

# 1. Configuration des chemins (Chemin relatif vers data/raw)
# os.getcwd() est dans /notebooks, on remonte pour aller dans /data/raw
dossier_raw = os.path.join(os.path.dirname(os.getcwd()), "data", "raw")
chemin_fichier = os.path.join(dossier_raw, "routes_idf.gpkg")

if not os.path.exists(dossier_raw):
    os.makedirs(dossier_raw)

# 2. Téléchargement et filtrage
print("Extraction des axes routiers (motorway, trunk, primary, secondary)...")
mes_axes = ["motorway", "trunk", "primary", "secondary"]

# On récupère les données
routes_idf = ox.features_from_place("Île-de-France, France", tags={"highway": mes_axes})

# On ne garde que les lignes
routes_idf = routes_idf[routes_idf.geometry.type.isin(['LineString', 'MultiLineString'])]

for col in routes_idf.columns:
    if routes_idf[col].apply(lambda x: isinstance(x, list)).any():
        routes_idf[col] = routes_idf[col].astype(str)

# Sauvegarde finale
routes_idf.to_file(chemin_fichier, driver="GPKG")

print(f"Extraction terminée avec succès.")
print(f"Fichier disponible pour les autres notebooks dans : data/raw/routes_idf.gpkg")

Extraction des axes routiers (motorway, trunk, primary, secondary)...


KeyboardInterrupt: 